# Building a FAISS-Based Vector Store: A Journey Through Data Processing and Visualization

In this notebook, you'll learn how to transform raw PDF documents into a searchable vector store using FAISS. We'll go on a journey where we:

1. **Read and extract text from PDF files.**
2. **Split the text into manageable chunks.**
3. **Display tokenization outputs from different tokenizers.**
4. **Generate embeddings from the text using a SentenceTransformer.**
5. **Store the embeddings in a FAISS index.**
6. **Project the embeddings into 2D space using UMAP for visualization.**
7. **Visualize the entire process on a scatter plot.**
8. **Incect your data into a prompt for a large language model**

In [1]:
import os
import tqdm
import glob
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings  # For generating embeddings for text chunks
import faiss
import pickle
import matplotlib.pyplot as plt
import umap.umap_ as umap
import numpy as np
from dotenv import load_dotenv
from groq import Groq


/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Reading Data from PDFs

First, we load PDF files from a directory, extract their text content, and combine it into one large text string.

In [2]:
### load the pdf from the path
glob_path = "data/*.pdf"
text = ""
for pdf_path in tqdm.tqdm(glob.glob(glob_path)):
    with open(pdf_path, "rb") as file:
        print(file)
        reader = PdfReader(file)
         # Extract text from all pages in the PDF
        text += " ".join(page.extract_text() for page in reader.pages if page.extract_text())

text[:50]

  0%|          | 0/9 [00:00<?, ?it/s]

<_io.BufferedReader name='data/Z_MB_Merkblatt_Verwendung_von_generativer_KI_in_Arbeiten.pdf'>


 11%|█         | 1/9 [00:00<00:04,  1.74it/s]

<_io.BufferedReader name='data/Z_RL_Richtlinie_KI_bei_Leistungsnachweisen.pdf'>


 22%|██▏       | 2/9 [00:00<00:03,  2.31it/s]

...

<_io.BufferedReader name='data/05_Checkliste_Sprachliche_Formale_Ausarbeitung.pdf'>
<_io.BufferedReader name='data/Schwerpunktthemen_fuer_Studenten.pdf'>


 89%|████████▉ | 8/9 [00:02<00:00,  3.30it/s]

<_io.BufferedReader name='data/02_Merkblatt_Vermeidung-von-Plagiaten_0916.pdf'>
<_io.BufferedReader name='data/W_MB_Merkblatt_Bachelorarbeit_BSc.pdf'>


100%|██████████| 9/9 [00:03<00:00,  2.57it/s]


'Z-MB-Merkblatt Verwendung von  \ngenerativer KI bei'

## 2. Splitting the Text into Chunks

Large texts can be difficult to work with. We use a text splitter, in this case [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/how_to/recursive_text_splitter/),  to break the full text into smaller, overlapping chunks. This helps preserve context when we later embed the text.

In [3]:
# Create a splitter: 2000 characters per chunk with an overlap of 200 characters
splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
# Split the extracted text into manageable chunks
chunks = splitter.split_text(text)

In [4]:
print(f"Total chunks: {len(chunks)}")
print("Preview of the first chunk:", chunks[0][:200])

Total chunks: 62
Preview of the first chunk: Z-MB-Merkblatt Verwendung von  
generativer KI bei Arbeiten  
Version:  1.2.0 gültig ab:  01.03.2025   Seite 1 von 5 
 Rektorat  
Ressort Bildung  
Verwendung von generativer KI bei Arbeiten  
Dieses 


## 3. Tokenizing the Text with Different Tokenizers

Before embedding, it's insightful to see how different tokenizers break up our text. Here, we use the tokenizer from the SentenceTransformer model (see [SentenceTransformersTokenTextSplitter](https://python.langchain.com/api_reference/text_splitters/sentence_transformers/langchain_text_splitters.sentence_transformers.SentenceTransformersTokenTextSplitter.html#sentencetransformerstokentextsplitter)).

In [5]:
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=128, model_name="paraphrase-multilingual-MiniLM-L12-v2")

In [6]:
token_split_texts = []
for text in chunks:
    token_split_texts += token_splitter.split_text(text)

print(f"\nTotal chunks: {len(token_split_texts)}")
print(token_split_texts[0])


Total chunks: 254
Z-MB-Merkblatt Verwendung von generativer KI bei Arbeiten Version: 1.2.0 gültig ab: 01.03.2025 Seite 1 von 5 Rektorat Ressort Bildung Verwendung von generativer KI bei Arbeiten Dieses Merkblatt basiert auf : − Richtlinie KI bei Leistungsnachweisen − ergänzender Anhang 1. Grundsatz Grundsätzlich gilt, dass die Verwendung von generativen KI -Tools bei Arbeiten zu de- klarieren ist. Dieses Merkblatt hilft Ihnen Schritt -für-Schritt bei der korrekten Umset- zung. 2. Pflichten im Umgang Ihre Pflichten im Umgang mit generativer KI − Sie


In [7]:
model_name = "paraphrase-multilingual-MiniLM-L12-v2"
model = SentenceTransformer(model_name)
tokenized_chunks = []
for i, text in enumerate(token_split_texts[:10]):
    # Tokenize each chunk
    encoded_input = model.tokenizer(text, padding=True, truncation=True, max_length=128, return_tensors='pt')
    # Convert token IDs back to tokens
    tokens = model.tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][0].tolist())
    tokenized_chunks.append(tokens)
    print(f"Chunk {i}: {tokens}")

Chunk 0: ['<s>', '▁Z', '-', 'MB', '-', 'Mer', 'k', 'blatt', '▁Verwendung', '▁von', '▁genera', 'ti', 'ver', '▁KI', '▁bei', '▁Arbeiten', '▁Version', ':', '▁1.', '2.0', '▁gültig', '▁ab', ':', '▁01.', '03.20', '25', '▁Seite', '▁1', '▁von', '▁5', '▁Rektor', 'at', '▁Res', 'sort', '▁Bildung', '▁Verwendung', '▁von', '▁genera', 'ti', 'ver', '▁KI', '▁bei', '▁Arbeiten', '▁Dieses', '▁Merk', 'blatt', '▁bas', 'iert', '▁auf', '▁:', '▁−', '▁', 'Richtlinie', '▁KI', '▁bei', '▁Leistungs', 'nach', 'weise', 'n', '▁−', '▁ergänzen', 'der', '▁An', 'hang', '▁1.', '▁Grund', 'satz', '▁Grundsätzlich', '▁gilt', ',', '▁dass', '▁die', '▁Verwendung', '▁von', '▁genera', 'tive', 'n', '▁KI', '▁-', 'Tool', 's', '▁bei', '▁Arbeiten', '▁zu', '▁de', '-', '▁klar', 'ieren', '▁ist', '.', '▁Dieses', '▁Merk', 'blatt', '▁hilft', '▁Ihnen', '▁Schritt', '▁-', 'für', '-', 'Sch', 'rit', 't', '▁bei', '▁der', '▁korrekt', 'en', '▁Um', 'set', '-', '▁', 'zung', '.', '▁2.', '▁Pflicht', 'en', '▁im', '▁Umgang', '▁Ihre', '▁Pflicht', 'en', '▁im'

In [8]:
model_name = "Sahajtomar/German-semantic"
model = SentenceTransformer(model_name)
tokenized_chunks = []
for i, text in enumerate(token_split_texts[:10]):
    # Tokenize each chunk
    encoded_input = model.tokenizer(text, padding=True, truncation=True, max_length=128, return_tensors='pt')
    # Convert token IDs back to tokens
    tokens = model.tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][0].tolist())
    tokenized_chunks.append(tokens)
    print(f"Chunk {i}: {tokens}")

Chunk 0: ['[CLS]', 'Z', '-', 'MB', '-', 'Merk', '##blatt', 'Verwendung', 'von', 'gener', '##ativer', 'K', '##I', 'bei', 'Arbeiten', 'Version', ':', '1', '.', '2', '.', '0', 'gültig', 'ab', ':', '01', '.', '03', '.', '202', '##5', 'Seite', '1', 'von', '5', 'Rektor', '##at', 'Ressort', 'Bildung', 'Verwendung', 'von', 'gener', '##ativer', 'K', '##I', 'bei', 'Arbeiten', 'Dieses', 'Merk', '##blatt', 'basiert', 'auf', ':', '−', 'Richtlinie', 'K', '##I', 'bei', 'Leistungs', '##nach', '##weisen', '−', 'ergänzen', '##der', 'Anhang', '1', '.', 'Grundsatz', 'Grundsätzlich', 'gilt', ',', 'dass', 'die', 'Verwendung', 'von', 'gener', '##ativen', 'K', '##I', '-', 'Tool', '##s', 'bei', 'Arbeiten', 'zu', 'de', '-', 'klar', '##ieren', 'ist', '.', 'Dieses', 'Merk', '##blatt', 'hilft', 'Ihnen', 'Schritt', '-', 'für', '-', 'Schritt', 'bei', 'der', 'korrekt', '##en', 'Um', '##set', '-', 'zun', '##g', '.', '2', '.', 'Pflichten', 'im', 'Umgang', 'Ihre', 'Pflichten', 'im', 'Umgang', 'mit', 'gener', '##ativer',

## 4. Generating Embeddings for Each Chunk

Now we convert each text chunk into a numerical embedding that captures its semantic meaning. These embeddings will be used for similarity search.

In [9]:
embeddings = HuggingFaceEmbeddings(model_name="paraphrase-multilingual-MiniLM-L12-v2")
chunk_embeddings = model.encode(token_split_texts, convert_to_numpy=True)

/tmp/ipykernel_68843/4090508586.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="paraphrase-multilingual-MiniLM-L12-v2")


: 

## 5. Building a FAISS Vector Store

FAISS is a powerful library for efficient similarity search. Here, we build an index from our embeddings. Remember, FAISS only stores the numerical vectors so we must keep our original text mapping separately.

In [ ]:
d = chunk_embeddings.shape[1]
print(d)

In [ ]:
index = faiss.IndexFlatL2(d)
index.add(chunk_embeddings)
print("Number of embeddings in FAISS index:", index.ntotal)

In [ ]:
if not os.path.exists('faiss'):
    os.makedirs('faiss')
    
faiss.write_index(index, "faiss/faiss_index.index")
with open("faiss/chunks_mapping.pkl", "wb") as f:
    pickle.dump(chunks, f)

In [ ]:
index_2 = faiss.read_index("faiss/faiss_index.index")
with open("faiss/chunks_mapping.pkl", "rb") as f:
    token_split_texts_2 = pickle.load(f)
print(len(token_split_texts_2))
print(len(token_split_texts))

## 6. Projecting Embeddings with UMAP

To visualize high-dimensional embeddings, we use UMAP to project them into 2D space. You can project both the entire dataset and individual query embeddings.

In [ ]:
# Fit UMAP on the full dataset embeddings
umap_transform = umap.UMAP(random_state=0, transform_seed=0).fit(chunk_embeddings)

def project_embeddings(embeddings, umap_transform):
    """
    Project a set of embeddings using a pre-fitted UMAP transform.
    """
    umap_embeddings = np.empty((len(embeddings), 2))
    for i, embedding in enumerate(tqdm.tqdm(embeddings, desc="Projecting Embeddings")):
        umap_embeddings[i] = umap_transform.transform([embedding])
    return umap_embeddings


In [ ]:
# Project the entire dataset embeddings
projected_dataset_embeddings = project_embeddings(chunk_embeddings, umap_transform)
print("Projected dataset embeddings shape:", projected_dataset_embeddings.shape)

## 7. Querying the Vector Store and Projecting Results

We now define a retrieval function that takes a text query, embeds it, and searches our FAISS index for similar documents. We then project these result embeddings with UMAP.
"""

In [ ]:
def retrieve(query, k=5):
    """
    Retrieve the top k similar text chunks and their embeddings for a given query.
    """
    query_embedding = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, k)
    retrieved_texts = [token_split_texts[i] for i in indices[0]]
    retrieved_embeddings = np.array([chunk_embeddings[i] for i in indices[0]])
    return retrieved_texts, retrieved_embeddings, distances[0]

In [ ]:
query = "KI während der Bachelorarbeit"
results, result_embeddings, distances = retrieve(query, k=3)
print("Retrieved document preview:")
print(results[0][:300])

In [ ]:
# Project the result embeddings
projected_result_embeddings = project_embeddings(result_embeddings, umap_transform)

# Also embed and project the original query for visualization
query_embedding = model.encode([query], convert_to_numpy=True)
project_original_query = project_embeddings(query_embedding, umap_transform)

## 8. Visualizing the Results

Finally, we create a scatter plot to visualize the entire dataset, the retrieved results, and the original query in 2D space.

In [ ]:

def shorten_text(text, max_length=15):
    """Shortens text to max_length and adds an ellipsis if shortened."""
    return (text[:max_length] + '...') if len(text) > max_length else text

plt.figure()

# Scatter plots
plt.scatter(projected_dataset_embeddings[:, 0], projected_dataset_embeddings[:, 1],
            s=10, color='gray', label='Dataset')
plt.scatter(projected_result_embeddings[:, 0], projected_result_embeddings[:, 1],
            s=100, facecolors='none', edgecolors='g', label='Results')
plt.scatter(project_original_query[:, 0], project_original_query[:, 1],
            s=150, marker='X', color='r', label='Original Query')

# If results is a list of texts, iterate directly
for i, text in enumerate(results):
    if i < len(projected_result_embeddings):
        plt.annotate(shorten_text(text),
                     (projected_result_embeddings[i, 0], projected_result_embeddings[i, 1]),
                     fontsize=8)

# Annotate the original query point
original_query_text = 'Welche hilfsmittel sind erlaubt?'  # Replace with your actual query text if needed
original_query_text = 'Wieviele Seiten muss die Arbeit sein?'  # Replace with your actual query text if needed

plt.annotate(shorten_text(original_query_text),
             (project_original_query[0, 0], project_original_query[0, 1]),
             fontsize=8)

plt.gca().set_aspect('equal', 'datalim')
plt.title('Visualization')
plt.legend()
plt.show()


---

# 📝 Task: Semantic Retrieval-Augmented Question Answering Using Groq LLM

## Objective
Implement a question-answering system that:
1. Retrieves the most semantically relevant text passages to a user query.
2. Constructs a natural language prompt based on the retrieved content.
3. Uses a large language model (LLM) hosted by Groq to generate an answer.

---

## Task Breakdown

### 1. Embedding-Based Semantic Retrieval
- Use the `SentenceTransformer` model `"Sahajtomar/German-semantic"` to encode a user query into a dense vector embedding.
- Perform a nearest-neighbor search in a prebuilt FAISS index to retrieve the top-**k** similar text chunks. You can **use the prebuilt FAISS form above**.


### 2. LLM Prompt Construction and Query Answering
- Build the prompt:
  - Using the retrieved text chunks, concatenates the results into a context block.
  - Builds a **prompt** asking the LLM to answer the question using that context.
  - Sends the prompt to the **Groq LLM API** (`llama-3.3-70b-versatile`) and returns the response.

### 3. User Query Execution
- An example query (`"What is the most important factor in diagnosing asthma?"`) is used to demonstrate the pipeline.
- The final answer from the LLM is printed.


## Tools & Models Used
- **SentenceTransformers** (`Sahajtomar/German-semantic`) for embedding generation.
- **FAISS** for efficient vector similarity search.
- **Groq LLM API** (`llama-3.3-70b-versatile`) for generating the final response.


In [2]:
import os
from dotenv import load_dotenv

# Lade .env aus dem env-Unterordner
load_dotenv(dotenv_path="env/.env")

# Zugriff auf die Variable
groq_api_key = os.getenv("GROQ_API_KEY")

print("GROQ API Key:", groq_api_key)  # Testausgabe (nur zum Testen!)


GROQ API Key: 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


In [3]:
import requests

# 1. Frage formulieren
query = "Was ist der wichtigste Faktor bei der Diagnose von Asthma?"

# 2. LLM-Embedding-Modell laden
model = SentenceTransformer("Sahajtomar/German-semantic")
query_embedding = model.encode([query], convert_to_numpy=True)

# 3. FAISS-Index und zugehörige Chunks laden
index = faiss.read_index("faiss/faiss_index.index")
with open("faiss/chunks_mapping.pkl", "rb") as f:
    token_split_texts_2 = pickle.load(f)

# 4. Top-k ähnliche Chunks abrufen
k = 5
D, I = index.search(query_embedding, k)

# Schutz gegen ungültige Indizes
retrieved_chunks = [token_split_texts_2[i] for i in I[0] if i < len(token_split_texts_2)]

# 5. Prompt konstruieren
context = "\n\n".join(retrieved_chunks)
prompt = f"""
Du bist ein medizinischer Assistent. Beantworte die folgende Frage **nur** basierend auf dem gegebenen Kontext.

Kontext:
{context}

Frage: {query}
Antwort:
"""

# 6. Anfrage an das Groq-LLM
url = "https://api.groq.com/openai/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {groq_api_key}",
    "Content-Type": "application/json"
}
payload = {
    "model": "llama3-70b-8192",
    "messages": [
        {"role": "system", "content": "Du bist ein hilfreicher medizinischer Assistent auf Deutsch."},
        {"role": "user", "content": prompt}
    ],
    "temperature": 0.3
}

response = requests.post(url, headers=headers, json=payload)

# Troubleshooting
print("Antwort der API:", response.status_code)
print(response.json())

answer = response.json()["choices"][0]["message"]["content"]

# 7. Ausgabe
print("Antwort des LLM:")
print(answer)

Antwort der API: 200
{'id': 'chatcmpl-46d45619-d366-48af-8f50-1b90dd08ab07', 'object': 'chat.completion', 'created': 1747872066, 'model': 'llama3-70b-8192', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Der wichtigste Faktor bei der Diagnose von Asthma ist die Anamnese, also die Erfassung der Krankheitsgeschichte und der Symptome durch Befragung des Patienten.'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'queue_time': 0.09352358699999999, 'prompt_tokens': 88, 'prompt_time': 0.002425165, 'completion_tokens': 46, 'completion_time': 0.131428571, 'total_tokens': 134, 'total_time': 0.133853736}, 'usage_breakdown': {'models': None}, 'system_fingerprint': 'fp_dd4ae1c591', 'x_groq': {'id': 'req_01jvtmpacre0d8kaq9k32cxk9q'}}
Antwort des LLM:
Der wichtigste Faktor bei der Diagnose von Asthma ist die Anamnese, also die Erfassung der Krankheitsgeschichte und der Symptome durch Befragung des Patienten.
